In [ ]:
#!pip install openai
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 294.6 kB/s eta 0:00:00


In [ ]:
!pip install torch
import torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import numpy as np
import openai
import pandas as pd
import json
#import tensorflow as tf
#from tensorflow import keras
from google.colab import drive
#from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
#from datasets import Dataset, load_metric
from sklearn.metrics import accuracy_score

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/266/booksummaries.txt'

df = pd.read_csv(file_path, delimiter='\t', encoding='utf-8', header=None)

# Define the column names
df.columns = ['ID', 'MID', 'Title', 'Author', 'Date', 'Genres', 'Summary']

# Function to parse JSON safely
def parse_json_safe(json_str):
    try:
        if isinstance(json_str, str):  # Check if the value is a string
            return json.loads(json_str)
        else:
            return {}  # Return an empty dict if not a string
    except json.JSONDecodeError:
        return {}  # Return an empty dict if JSON decoding fails

# Apply the function to the 'Genres' column
df['Genres'] = df['Genres'].apply(parse_json_safe)

# Function to extract genres from the parsed JSON
def extract_genres(genre_dict):
    if isinstance(genre_dict, dict):
        return list(genre_dict.values())
    return []

# Apply the function to the 'Genres' column to get lists of genres
df['genre_list'] = df['Genres'].apply(extract_genres)

# Drop records missing genre
df = df[~(df['genre_list'].str.len() == 0)]
df = df.reset_index(drop=True)

df = df.drop(columns = ['Genres', 'MID'])

# with data cleaned, trim for testing
random_seed = 42

# Sample 5000 records with the random seed
df = df.sample(5000, random_state=random_seed)

# Unique genres
flattened_genres = [genre for sublist in df['genre_list'] for genre in sublist]

df.head(4)
#df.iloc[0]


,ID,Title,Author,Date,Summary,genre_list
8602,12200712,Bitter Lemons,Lawrence Durrell,NaN,"The book is alternately comic and serious, ch...","[Reference, Non-fiction]"
11624,24694031,Touch the Dark,NaN,2006-06,Cassandra Palmer is a clairvoyant. She can se...,[Urban fantasy]
8347,11494990,Starcross,Philip Reeve,2007-10,"Protagonist Arthur (""Art"") Mumby and his olde...","[Science Fiction, Children's literature, Specu..."
8965,13340278,Wintle's Wonders,Noel Streatfeild,1957,"As the book begins, Rachel and her adopted si...",[Children's literature]


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(df['Summary'], df, test_size=0.2, random_state=9)

In [ ]:
len(ytest)

1000

In [ ]:
ytest.head(1)

,ID,Title,Author,Date,Summary,genre_list
9968,16932853,Daniel's Story,Carol Matas,1993,Daniel barely remembers leading a normal life...,[Children's literature]


In [ ]:
# Set your OpenAI API key
openai.api_key = ##### FOR PRIVACY I DO NOT INCLUDE... ####

def classify_genres(summary, genre_list, delay=1):
    """
    Classify the genres of a given book summary using OpenAI API.

    Args:
    summary (str): The book summary to classify.
    genre_list (list): List of possible genres.

    Returns:
    str: Comma-separated list of genres predicted by the API.
    """
    prompt = (f"Given the following book summary, list the appropriate genres from this list: {', '.join(genre_list)}.\n\n"
              f"Summary: {summary}")

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use the appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=0.3
    )

    genres = response.choices[0].message['content'].strip()
    return genres

def process_summaries(input_df, genre_list):
    """
    Process a dataframe of book summaries and classify genres.

    Args:
    input_file (str): Path to the input CSV file containing book summaries.
    output_file (str): Path to the output CSV file to save results.
    genre_list (list): List of possible genres.
    """
    # Read the dataset
    df = input_df

    # Ensure the column containing summaries is named 'Summary'
    if 'Summary' not in df.columns:
        raise ValueError("Input file must contain a 'Summary' column.")

    # Classify genres for each summary
    df['Predictions'] = df['Summary'].apply(lambda x: classify_genres(x, genre_list))

    # Output new df
    return df


In [ ]:
all_genres = list(set(flattened_genres))

# $0.07 for 100 examples


In [ ]:
# Example usage
input_df = ytest  # Use the cleaned 1,000 ytest values
genre_list = all_genres

process_summaries(input_df, genre_list)

,ID,Title,Author,Date,Summary,genre_list,Predictions
9968,16932853,Daniel's Story,Carol Matas,1993,Daniel barely remembers leading a normal life...,[Children's literature],"Genres: Historical fiction, War novel, Holocau..."
2254,1581133,Isle of the Dead,Roger Zelazny,1969,"Sandow is jolted from his wealthy, indolent l...","[Science Fiction, Speculative fiction]",- Science Fiction\n- Space opera\n- Supernatural
4215,3901027,Madame de Mauves,Henry James,NaN,Outside Paris a wealthy American man named Lo...,[Novella],- Historical fiction\n- Romance novel
5740,5667805,Soldiers Live,Glen Cook,2000-07,"In this climactic conclusion to the ""Glitteri...","[Science Fiction, Speculative fiction, Fantasy...",The appropriate genres for the given book summ...
2788,2165536,Jonathan Troy,Edward Abbey,1954,Many of the other characters in the book refe...,[Western],- Bildungsroman\n- Historical fiction\n- Comin...
...,...,...,...,...,...,...,...
12262,29449367,People's Republic,Robert Muchamore,NaN,People's Republic begins with us being introd...,"[Thriller, Spy fiction, Children's literature]",- Espionage\n- Young adult literature\n- Thril...
7302,8746994,Step on a Crack,M. T. Coffin,2007-02-06,"When a beloved former First Lady dies, an ela...","[Thriller, Crime Fiction, Speculative fiction,...",Genres that apply to the given book summary ar...
2487,1859415,Nobody's Buddy,John A. Moroso,1936,"Sweet story of a boy and his dog, and their a...",[Children's literature],"Genres: Adventure, Children's literature"
8902,13193574,The Nobodies,NaN,2005-06,Fern Drudger has the ability to shake things ...,"[Children's literature, Fantasy]",- Fantasy\n- Young adult literature\n- Adventure


In [ ]:
input_df.head(1)

,ID,Title,Author,Date,Summary,genre_list,Predictions
9968,16932853,Daniel's Story,Carol Matas,1993,Daniel barely remembers leading a normal life...,[Children's literature],"Genres: Historical fiction, War novel, Holocau..."


In [ ]:
def clean_genre_record(record):
    if record.startswith("Genres: "):
        # Remove the prefix 'Genres: ' and trailing commas, then split by comma
        genres = record.replace('Genres: ', '').strip().split(',')
    else:
        # Split by new lines and strip each line to remove leading/trailing spaces
        genres = [genre.strip() for genre in record.split('\n') if genre.strip()]
        # Remove leading hyphens and spaces from each genre
        genres = [genre.lstrip('- ').strip() for genre in genres]

    # Strip each genre to remove leading/trailing spaces and filter out any empty strings
    cleaned_genres = [genre.strip() for genre in genres if genre.strip()]
    return cleaned_genres

# Apply the function to the column
input_df['Predictions'] = input_df['Predictions'].apply(clean_genre_record)

# save a copy as output_df for clarity
output_df = input_df


In [ ]:
output_df.iloc[0]

,9968
ID,16932853
Title,Daniel's Story
Author,Carol Matas
Date,1993
Summary,"Daniel barely remembers leading a normal life before the Nazis came to power in 1933. He can still picture once being happy and safe, but memories and days are fading away as he and his family face the dangers threatening Jews in Hitler's Germany the late 1930s. No longer able to practice their religion, vote, own property, or work, Daniel's family is forced from their home in Frankfurt. First, they are deported to the ghetto in Lodz,Poland, and then to Auschwitz (the Nazi extermination camp). He survives the torments of the concentration camp and is transported to Buchenwald. Daniel endures to witness the camp's liberation in 1945. Though many around him lose hope in the face of such terror, Daniel, supported by his courageous family, struggles for survival. Yet he manages to retain his life, hope and dignity through the horrors of Hitler's Final Solution."
genre_list,[Children's literature]
Predictions,"[Historical fiction, War novel, Holocaust literature]"


Note that even though I specified to use the list of genres which are the flattened genres list from input_df['genre_list'], OpenAI included other genres,such as "Holocaust literature". Because of the wider genre vocabulary, we don't compute F1 scores, but rely on the cosine similarity.

## Cosine Similarity

To compare df['genre_list'] with df['Predictions']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Function to get embeddings for a list of genres
def get_genre_embedding(genre):
    inputs = tokenizer(genre, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

all_genres = list(set(flattened_genres))

# Get embeddings for all unique genres
# genre_embeddings = get_genre_embeddings(all_genres)

def get_genres_embeddings(genres_list):
    return [get_genre_embedding(genre) for genre in genres_list]


# The inverse_transform genre lists are actually 1-D arrays of tuples. We need to convert to 2-D list of lists.
def listpadding(list_o_tuples):
    # Determine the maximum tuple length
    max_len = max(len(t) for t in list_o_tuples)

    # Pad the tuples to have the same length
    padded_genres = [t + ('',) * (max_len - len(t)) for t in list_o_tuples]

    # Convert to a 2-D numpy array
    return np.array(padded_genres)


##### Since the predictions here are already lists of words (rather than one-hot vectors), we don't need the inverse-binarizer.

In [ ]:
# Compute cosine similarities for each pair of actual and predicted genres
def calculate_cosine_similarities(y_actual, y_pred):
    similarities = []
    for actual_genres, pred_genres in zip(y_actual, y_pred):
        actual_embeddings = get_genres_embeddings(actual_genres)
        pred_embeddings = get_genres_embeddings(pred_genres)

        # Calculate pairwise cosine similarity
        if actual_embeddings and pred_embeddings:
            similarity_matrix = cosine_similarity(actual_embeddings, pred_embeddings)
            avg_similarity = np.mean(similarity_matrix)
        else:
            avg_similarity = 0.0
        similarities.append(avg_similarity)
    return similarities

# Calculate cosine similarities between the text vectors of actual and predicted genres
cosine_similarities = calculate_cosine_similarities(output_df['genre_list'][0:51], output_df['Predictions'][0:51])

# Print the cosine similarities
for idx, similarity in enumerate(cosine_similarities):
    print(f"Document {idx + 1}: Cosine Similarity = {similarity:.4f}")

Document 1: Cosine Similarity = 0.6933
Document 2: Cosine Similarity = 0.7726
Document 3: Cosine Similarity = 0.7490
Document 4: Cosine Similarity = 0.4270
Document 5: Cosine Similarity = 0.5396
Document 6: Cosine Similarity = 0.8221
Document 7: Cosine Similarity = 0.8409
Document 8: Cosine Similarity = 0.7998
Document 9: Cosine Similarity = 0.7822
Document 10: Cosine Similarity = 0.8590
Document 11: Cosine Similarity = 0.7436
Document 12: Cosine Similarity = 0.8979
Document 13: Cosine Similarity = 0.8154
Document 14: Cosine Similarity = 0.7486
Document 15: Cosine Similarity = 0.7129
Document 16: Cosine Similarity = 0.6389
Document 17: Cosine Similarity = 0.8242
Document 18: Cosine Similarity = 0.8906
Document 19: Cosine Similarity = 0.7070
Document 20: Cosine Similarity = 0.7801
Document 21: Cosine Similarity = 0.7278
Document 22: Cosine Similarity = 0.8219
Document 23: Cosine Similarity = 0.3939
Document 24: Cosine Similarity = 0.8148
Document 25: Cosine Similarity = 0.7284
Document 

### Average Cosine Similarities

In [ ]:
def calculate_cosine_similarities(y_actual, y_pred):
    all_similarities = []
    for actual_genres, pred_genres in zip(y_actual, y_pred):
        actual_embeddings = get_genres_embeddings(actual_genres)
        pred_embeddings = get_genres_embeddings(pred_genres)

        # Calculate pairwise cosine similarity
        if actual_embeddings and pred_embeddings:
            similarity_matrix = cosine_similarity(actual_embeddings, pred_embeddings)
            avg_similarity = np.mean(similarity_matrix)
        else:
            avg_similarity = 0.0
        all_similarities.append(avg_similarity)

    # Calculate and return the overall average similarity
    overall_avg_similarity = np.mean(all_similarities)
    return overall_avg_similarity

# Calculate overall cosine similarity
overall_cosine_similarity = calculate_cosine_similarities(output_df['genre_list'], output_df['Predictions'])

# Print the overall cosine similarity
print(f"Overall Cosine Similarity = {overall_cosine_similarity:.4f}")

Overall Cosine Similarity = 0.7228


In [ ]:
decoded_labels = output_df['Predictions']

# Count the number of records with empty tuples
count_empty = sum(len(labels) == 0 for labels in decoded_labels)
print(count_empty)

0


In [ ]:
print(output_df['Predictions'].iloc[0])
print(output_df['genre_list'].iloc[0])
calculate_cosine_similarities(output_df['Predictions'].iloc[0],output_df['genre_list'].iloc[0])

['Historical fiction', 'War novel', 'Holocaust literature']
["Children's literature"]


0.8222667

In [ ]:
print(output_df['Predictions'].iloc[30])
print(output_df['genre_list'].iloc[30])
calculate_cosine_similarities(output_df['Predictions'].iloc[30],output_df['genre_list'].iloc[30])

['Fantasy', 'Mythology', 'Native American Literature', 'Trickster Fiction']
['Postmodernism']


0.85325277

In [ ]:
print(output_df['Predictions'].iloc[50])
print(output_df['genre_list'].iloc[50])
calculate_cosine_similarities(output_df['Predictions'].iloc[50],output_df['genre_list'].iloc[50])

['Historical fiction', 'Literary fiction']
['Biography', 'Drama', 'Historical romance']


0.8306809